<a href="https://colab.research.google.com/github/Fredherve/Asset-Management/blob/main/gestion_active_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install dash

In [5]:
import dash
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# Téléchargement des données de prix
tickers = ['MSFT', 'NVDA', 'SPY', 'TLT', 'GLD', 'UUP', 'DBC', 'VWO', 'GBTC', 'USO', 'PALL']
start_date = '2019-01-01'
end_date = '2024-01-01'
prices = yf.download(tickers, start=start_date, end=end_date)['Close']

# Préparation des données pour les rendements mensuels
monthly_prices = prices.resample('ME').last()
monthly_returns = monthly_prices.pct_change()

# Calcul des rendements totaux et annualisés
total_returns = (prices.iloc[-1] / prices.iloc[0]) - 1
annualized_returns = (1 + total_returns) ** (1/5) - 1
total_returns_percentage = total_returns * 100
annualized_returns_percentage = annualized_returns * 100

# Création de l'application Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard d'Analyse des Actifs Financiers"),

    dcc.Dropdown(
        id='ticker-selection',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value=tickers,
        multi=True
    ),

    dcc.Graph(id='prices_chart'),

    html.Div(
        [
            dcc.Graph(id='correlation_matrix'),
            dcc.Graph(id='covariance_matrix')
        ],
        style={
            'display': 'flex',
            'gap': '20px',
            'overflowX': 'auto',  # Défilement horizontal si besoin
            'padding': '10px 0',  # Espace vertical
            'minWidth': '1200px',  # Largeur minimale totale (2x600px)
            'boxSizing': 'border-box'
        }
    ),

    html.Div(
        [
            html.Div(
                [html.H2("Rendements des Actifs"), html.Div(id='returns_output')],
                style={'flex': '1'}  # Correction
            ),
            html.Div(
                [html.H2("Poids Optimaux"), html.Div(id='optimal_weights_output')],
                style={'flex': '1'}  # Correction
            )
        ],
        style={'display': 'flex', 'gap': '20px'}
    ),

    html.Div(
        [
            # Colonne Graphique (3x plus large)
            html.Div(
                [
                    html.H2("Security Market Line (SML) et Portefeuille de Marché"),
                    dcc.Graph(id='sml_chart', style={'height': '500px'})
                ],
                style={
                    'flex': '2.5',  # Ratio 3:1 par rapport à la colonne de droite
                    'padding': '15px',
                    'background': '#f5f7fa',
                    'minWidth': '70%'  # Garantit un espace minimum dominant
                }
            ),

            # Colonne Informations (plus étroite)
            html.Div(
                [
                    html.H3("Informations sur le Portefeuille de Marché",
                           style={'marginBottom': '15px'}),
                    html.Div(id='market_portfolio_info',
                            style={
                                'padding': '15px',
                                'border': '1px solid #e1e4e8',
                                'borderRadius': '6px'
                            })
                ],
                style={
                    'flex': '1.5',
                    'padding': '15px',
                    'minWidth': '300px',  # Largeur minimum absolue
                    'maxWidth': '400px',  # Empêche l'expansion excessive
                    'background': '#ffffff'
                }
            )
        ],
        style={
            'display': 'flex',
            'gap': '25px',
            'margin': '20px 0',
            'borderRadius': '10px',
            'overflow': 'hidden'
        }
    )
])


@app.callback(
    [Output('prices_chart', 'figure'),
     Output('correlation_matrix', 'figure'),
     Output('covariance_matrix', 'figure'),
     Output('returns_output', 'children'),
     Output('optimal_weights_output', 'children'),
     Output('sml_chart', 'figure'),
     Output('market_portfolio_info', 'children')],
    [Input('ticker-selection', 'value')]
)
def update_graphs(selected_tickers):
    if not selected_tickers:
        return px.line(), px.imshow(), px.imshow(), "Aucun ticker sélectionné.", "Aucune donnée.", px.line(), ""

    # Filtrage des données en fonction des tickers sélectionnés
    filtered_prices = prices[selected_tickers]
    filtered_monthly_returns = monthly_returns[selected_tickers]
    filtered_annualized_returns = annualized_returns[selected_tickers]
    filtered_cov_matrix_annualized = filtered_monthly_returns.cov() * 12

    # Graphique des prix
    fig_prices = px.line(filtered_prices, title='Évolution des Prix des Actifs')

    # Matrice de corrélation
    corr_matrix = filtered_monthly_returns.corr()
    fig_corr = px.imshow(corr_matrix, labels=dict(color='Corrélation'), title="Matrice de Corrélation des Rendements")

    # Matrice de covariance
    fig_cov = px.imshow(filtered_cov_matrix_annualized, labels=dict(color='Covariance'), title="Matrice de Variance-Covariance Annualisée")

    # Optimisation du portefeuille
    def optimize_portfolio(target_return=0.15):
        mu = filtered_annualized_returns.values
        Sigma = filtered_cov_matrix_annualized.values
        n_assets = len(mu)
        weights = cp.Variable(n_assets)
        risk = cp.quad_form(weights, Sigma)
        constraints = [
            cp.sum(weights) == 1,
            weights @ mu >= target_return,
            weights >= 0,
            weights <= 1
        ]
        problem = cp.Problem(cp.Minimize(risk), constraints)

        try:
            problem.solve()
            optimal_weights = pd.Series(weights.value, index=selected_tickers)
            return problem.status, problem.value, optimal_weights
        except:
            return "Échec", None, pd.Series([None] * len(selected_tickers), index=selected_tickers)

    optimization_status, min_variance, optimal_weights = optimize_portfolio(target_return=0.15)

    # Construction de la SML et du portefeuille de marché
    risk_free_rate = 0.025
    mu_targets = np.linspace(0.02, 0.3, 100)
    risks = []

    def find_tangency_portfolio():
        max_slope = -np.inf
        tangency_weights, tangency_return, tangency_risk = None, None, None
        mu = filtered_annualized_returns.values
        Sigma = filtered_cov_matrix_annualized.values
        n_assets = len(mu)

        for target in mu_targets:
            weights = cp.Variable(n_assets)
            risk = cp.quad_form(weights, Sigma)
            constraints = [
                weights @ mu >= target,
                cp.sum(weights) == 1
            ]
            problem = cp.Problem(cp.Minimize(risk), constraints)
            problem.solve()

            portfolio_risk = np.sqrt(problem.value)
            risks.append(portfolio_risk)

            slope = (target - risk_free_rate) / portfolio_risk if portfolio_risk > 0 else -np.inf

            if slope > max_slope:
                max_slope = slope
                tangency_weights = weights.value
                tangency_return = target
                tangency_risk = portfolio_risk

        return tangency_weights, tangency_return, tangency_risk

    tangency_weights, tangency_return, tangency_risk = find_tangency_portfolio()

    # Tracé de la SML
    fig_sml = px.line(x=risks, y=mu_targets, labels={'x': 'Risque (Écart-type)', 'y': 'Rendement Espéré'}, title='Security Market Line (SML)')
    fig_sml.add_scatter(x=[tangency_risk], y=[tangency_return], mode='markers', marker=dict(color='red', size=10), name="Portefeuille de Marché")
    fig_sml.add_scatter(x=[0, tangency_risk], y=[risk_free_rate, tangency_return], mode='lines', line=dict(color='green', dash='dash'), name="Tangente (CAL)")

    # Affichage des données
    returns_table = dash_table.DataTable(
        data=pd.DataFrame({
            "Ticker": selected_tickers,
            "Rendements totaux (%)": total_returns_percentage[selected_tickers],
            "Rendements annualisés (%)": annualized_returns_percentage[selected_tickers]
        }).to_dict('records'),
        columns=[{'name': i, 'id': i} for i in ["Ticker", "Rendements totaux (%)", "Rendements annualisés (%)"]]
    )

    weights_table = dash_table.DataTable(
        data=pd.DataFrame(optimal_weights).reset_index().rename(columns={'index': 'Ticker', 0: 'Poids Optimal'}).to_dict('records'),
        columns=[{'name': i, 'id': i} for i in ['Ticker', 'Poids Optimal']]
    )

    market_info = html.Div([
        html.P(f"Rendement du Portefeuille de Marché : {tangency_return:.6f}"),
        html.P(f"Risque du Portefeuille de Marché : {tangency_risk:.6f}"),
        html.H4("Poids du Portefeuille de Marché"),
        dash_table.DataTable(data=pd.DataFrame({"Ticker": selected_tickers, "Poids": tangency_weights}).to_dict('records'),
                             columns=[{"name": i, "id": i} for i in ["Ticker", "Poids"]])
    ])

    return fig_prices, fig_corr, fig_cov, returns_table, weights_table, fig_sml, market_info


if __name__ == '__main__':
    app.run_server(debug=True)


[*********************100%***********************]  11 of 11 completed


<IPython.core.display.Javascript object>